In [33]:
import numpy as np
import nltk
import pandas as pd
from datasets import load_dataset
import re
import string
from bs4 import BeautifulSoup
import sklearn

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import scipy.stats as stats
#import spacy

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/alexacole/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexacole/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexacole/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
dataset = load_dataset('artem9k/ai-text-detection-pile')
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'id', 'text'],
        num_rows: 1392522
    })
})

In [35]:
df = pd.DataFrame.from_dict(dataset['train'])
df.head()

,source,id,text
0,human,0,12 Years a Slave: An Analysis of the Film Essa...
1,human,1,20+ Social Media Post Ideas to Radically Simpl...
2,human,2,2022 Russian Invasion of Ukraine in Global Med...
3,human,3,533 U.S. 27 (2001) Kyllo v. United States: The...
4,human,4,A Charles Schwab Corporation Case Essay\n\nCha...


In [36]:
df.source.value_counts()

source
human    1028146
ai        364376
Name: count, dtype: int64

## Preprocessing

In [37]:
# functions for preprocessing
def remove_urls(text):
    return re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", " ", text) # regex taken from https://www.geeksforgeeks.org/python-check-url-string/

def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_extra_whitespace(text):
    text = text.strip()
    text = " ".join(text.split())
    return text

def remove_stop_words(text):
    tokens = nltk.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words("english")
    tokens = [token for token in tokens if token not in stopwords]
    return " ".join(tokens)

def lemmatizer(text):
    tokens = nltk.word_tokenize(text)
    l = nltk.stem.WordNetLemmatizer()
    tokens = [l.lemmatize(token) for token in tokens]
    return " ".join(tokens)

def tokenize_pre_process(text): # for preprocessing using this link: https://spotintelligence.com/2022/12/21/nltk-preprocessing-pipeline/
    # tokenize
    tokens = nltk.word_tokenize(text)

    # remove stop words
    stopwords = nltk.corpus.stopwords.words("english")
    tokens = [token for token in tokens if token not in stopwords]

    # remove top 10% most frequent words 
    fdist = nltk.FreqDist(tokens)
    tokens = [token for token in tokens if fdist[token] < fdist.N() * 0.1]

    # stemming
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # eliminate punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    return tokens

In [38]:
def preprocess_text(text):
    # encoding to ascii
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # convert text to lower case
    text = text.lower()

    # remove html tags 
    text = remove_html(text)

    # remove urls 
    text = remove_urls(text)

    # remove extra whitespace
    text = remove_extra_whitespace(text)

    # remove stop words
    text = remove_stop_words(text)

    return text

In [39]:
def preprocess_text2(text):
    # encoding to ascii
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # convert text to lower case
    text = text.lower()

    # remove html tags 
    text = remove_html(text)

    # remove urls 
    text = remove_urls(text)

    # remove extra whitespace
    text = remove_extra_whitespace(text)

    # remove stop words
    text = remove_stop_words(text)

    # lemmatize words
    text = lemmatizer(text)
    
    return text

## Feature Engineering

### Count Vectorizer

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

In [41]:
vec = CountVectorizer(max_df=0.9,min_df=0.1)
X = vec.fit_transform(df.text[:1000])

In [42]:
X.toarray().shape

(1000, 615)

In [43]:
vec.get_feature_names_out()

array(['10', '11', '12', '15', '19', '2017', '2018', '2019', '2020',
       '2021', '2022', 'ability', 'able', 'about', 'above', 'access',
       'according', 'achieve', 'across', 'act', 'action', 'actions',
       'activities', 'activity', 'addition', 'additional', 'additionally',
       'address', 'affect', 'affected', 'affects', 'after', 'against',
       'age', 'al', 'all', 'allow', 'allowed', 'allows', 'almost',
       'already', 'also', 'although', 'always', 'america', 'american',
       'americans', 'among', 'an', 'analysis', 'another', 'any',
       'approach', 'approaches', 'appropriate', 'are', 'area', 'areas',
       'around', 'article', 'aspect', 'aspects', 'associated', 'at',
       'attention', 'author', 'available', 'avoid', 'back', 'based',
       'basis', 'be', 'became', 'because', 'become', 'becomes', 'been',
       'before', 'behavior', 'being', 'believe', 'benefits', 'best',
       'better', 'between', 'black', 'body', 'both', 'business', 'but',
       'by', 'can', 

In [44]:
vec2 = CountVectorizer(preprocessor=preprocess_text,max_df=0.9,min_df=0.1)
X2 = vec2.fit_transform(df.text[:1000])

In [45]:
vec2.get_feature_names_out()

array(['10', '11', '12', '15', '19', '2017', '2018', '2019', '2020',
       '2021', '2022', 'ability', 'able', 'access', 'according',
       'achieve', 'across', 'act', 'action', 'actions', 'activities',
       'activity', 'addition', 'additional', 'additionally', 'address',
       'affect', 'affected', 'affects', 'age', 'al', 'allow', 'allowed',
       'allows', 'almost', 'already', 'also', 'although', 'always',
       'america', 'american', 'americans', 'among', 'analysis', 'another',
       'approach', 'approaches', 'appropriate', 'area', 'areas', 'around',
       'article', 'aspect', 'aspects', 'associated', 'attention',
       'author', 'authors', 'available', 'avoid', 'back', 'based',
       'basis', 'became', 'become', 'becomes', 'behavior', 'believe',
       'benefits', 'best', 'better', 'body', 'business', 'care', 'case',
       'cases', 'cause', 'caused', 'causes', 'central', 'century',
       'certain', 'challenges', 'change', 'changes', 'characteristics',
       'children',

In [46]:
vec3 = CountVectorizer(preprocessor=preprocess_text2,max_df=0.9,min_df=0.1)
X3 = vec3.fit_transform(df.text[:1000])

In [47]:
vec3.get_feature_names_out()

array(['10', '11', '12', '15', '19', '2017', '2018', '2019', '2020',
       '2021', '2022', 'ability', 'able', 'access', 'according',
       'account', 'achieve', 'across', 'act', 'action', 'activity',
       'addition', 'additional', 'additionally', 'address', 'advantage',
       'affect', 'affected', 'age', 'aim', 'al', 'allow', 'allowed',
       'allows', 'almost', 'already', 'also', 'although', 'always',
       'america', 'american', 'among', 'amount', 'analysis', 'another',
       'application', 'approach', 'appropriate', 'area', 'around',
       'article', 'aspect', 'assessment', 'associated', 'attention',
       'attitude', 'author', 'authority', 'available', 'avoid', 'back',
       'background', 'based', 'basis', 'became', 'become', 'becomes',
       'behavior', 'being', 'belief', 'believe', 'benefit', 'best',
       'better', 'black', 'body', 'book', 'business', 'care', 'case',
       'cause', 'caused', 'center', 'central', 'century', 'certain',
       'challenge', 'chance', '

In [48]:
vec4 = CountVectorizer(preprocessor=preprocess_text2,max_df=0.9,min_df=0.1, ngram_range=(2,3))
X4 = vec4.fit_transform(df.text[:1000])
vec4.get_feature_names_out()

array(['al 2020', 'content introduction', 'covid 19', 'essay table',
       'essay table content', 'et al', 'et al 2020', 'research paper',
       'table content', 'table content introduction', 'united state',
       'work cited'], dtype=object)

### TFIDF

In [49]:
def tfidf(data_train, data_test):
    tfidf = TfidfVectorizer(preprocessor=preprocess_text2,max_df=0.9,min_df=0.1)
    train = tfidf.fit_transform(data_train)
    test = tfidf.transform(data_test)
    return train, test 

### Doc2Vec

In [50]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [51]:
def doc2vec(data):
    '''
    https://www.geeksforgeeks.org/doc2vec-in-nlp/
    '''
    
    tagged_data = [TaggedDocument(words=word_tokenize(doc.lower()),
                              tags=[str(i)]) for i,doc in enumerate(data)]
    # train the Doc2vec model
    model = Doc2Vec(vector_size=20,
                    min_count=2, epochs=50)
    model.build_vocab(tagged_data)
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)

    return model

def get_vectors(doc2vec_model, data):
    # get the document vectors
    return [doc2vec_model.infer_vector(word_tokenize(doc.lower())) for doc in data]

## Sampling from the dataset

### Preserving the original proportions of humans and ai

In [52]:
# Reporting the proportion of samples that are ai generated
print('Percent ai:', round(df[df['source'] == 'ai'].shape[0]/df[df['source'] == 'human'].shape[0]*100, 3))

# Taking a stratified sample of 0.1% of the data
# maintaining same proportions of human and ai samples
data = df.groupby('source', group_keys=False).apply(lambda x: x.sample(frac=0.001, random_state=0))

Percent ai: 35.44


/var/folders/q7/lxfgdwq942qdsd3_mlwlm7p00000gn/T/ipykernel_62893/875810769.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = df.groupby('source', group_keys=False).apply(lambda x: x.sample(frac=0.001, random_state=0))


### Randomly sampling 2000 samples each of human and ai sources

In [53]:
# Randomly sampling 2000 human and 2000 ai samples to create dataset with equal proportions
human = df[df.source == "human"]
human = human.sample(2000, random_state=0)
ai = df[df.source == "ai"].sample(2000, random_state=0)
equal = pd.concat([human,ai], ignore_index=True)
equal

,source,id,text
0,human,198049,Overview\n\nBatman and Psychology: A Dark and ...
1,human,12919,The Use of Psychedelic Drugs in Treating Depre...
2,human,979845,member the day like it was yesterday. \n My mo...
3,human,73499,"Legislative Branch Power, Its Limits and Expan..."
4,human,44380,Growth and Fall of Vader Corporation Report\n\...
...,...,...,...
3995,ai,1052501,"""Learn Python the Hard Way"" by Zed Shaw\n\t\t..."
3996,ai,1277472,Cape Town - The South African Humanist Associa...
3997,ai,1086015,"We are only days away from the 2016 NFL Draft,..."
3998,ai,1338694,The number of people infected with Zika virus ...


# Run Models

In [54]:
def CI(metric, confidence):
    a,b = stats.t.interval(confidence, 
                         len(metric)-1, 
                         loc=metric.mean(), 
                         scale=metric.std(ddof=1)/np.sqrt(len(metric)))
    return a,b

In [55]:
def evaluation(X_train, X_test, t_train, t_test, model, model_name, confidence=0.95, scoring='accuracy'):

    y_train = model.predict(X_train)
    y_test = model.predict(X_test)
    
    scores = cross_val_score(model,
                             X_train, 
                             t_train, 
                             scoring=scoring, 
                             cv=KFold(10, shuffle=True, random_state=0))
    
    a,b = CI(scores, confidence)
    
    print(f'==================={model_name} Performance=====================')
    print('95% CI = [', a, b, ']')
    print('Train: ', classification_report(t_train, y_train))
    print('Test: ', classification_report(t_test, y_test))

In [62]:
def run_models(data, features):
    # Convert labels into numeric
    t = data.source
    d = {'human' : 0, 'ai' : 1}
    t = t.map(d, na_action='ignore')

    # Split the data into training and test sets
    X_train, X_test, t_train, t_test = train_test_split(np.array(data.text), t, test_size=0.2, random_state=0)
    
    if features == 'tfidf':
        X_train, X_test = tfidf(X_train,X_test)
        X_train = X_train.toarray()
        X_test = X_test.toarray()
    elif features == 'doc2vec':
        train_prep = []
        test_prep = []
        for i in range(len(X_train)):
            train_prep += [preprocess_text2(X_train[i])]
        for i in range(len(X_test)):
            test_prep += [preprocess_text2(X_test[i])]
        
        m = doc2vec(train_prep)
        X_train = np.array(get_vectors(m, train_prep))
        X_test = np.array(get_vectors(m, test_prep))

    # Naive Bayes -- need to scale for doc2vec
    if features == 'tfidf':
        mnb_pipe = Pipeline([('mnb', MultinomialNB())])
    elif features == 'doc2vec':
        mnb_pipe = Pipeline([('scaler', MinMaxScaler()),('mnb', MultinomialNB())])
    mnb_param = {'mnb__alpha': [0.2, 0.4, 0.6, 0.8, 1], 'mnb__force_alpha': [True, False]}
    
    
    gs1 = GridSearchCV(mnb_pipe, 
                   param_grid=mnb_param,
                   cv=KFold(n_splits=5, shuffle=True, random_state=0), 
                   scoring='accuracy',
                   verbose=1, 
                   n_jobs=-1, 
                   refit=True)

    gs1.fit(X_train, t_train)
    mnb = gs1.best_estimator_
    evaluation(X_train, X_test, t_train, t_test, mnb, 'MNB')
    
    # Decision Tree
    dt_pipe = Pipeline([('dt', DecisionTreeClassifier())])
    # hyperparameters selected taken from https://www.geeksforgeeks.org/how-to-tune-a-decision-tree-in-hyperparameter-tuning/
    dt_param = {'dt__max_depth': [10, 20, 30, None],'dt__min_samples_split': [2, 5, 10],'dt__min_samples_leaf': [1, 2, 4]}
    
    gs2 = GridSearchCV(dt_pipe, 
                   param_grid=dt_param,
                   cv=KFold(n_splits=5, shuffle=True, random_state=0), 
                   scoring='accuracy',
                   verbose=1, 
                   n_jobs=-1, 
                   refit=True)

    gs2.fit(X_train, t_train)
    dt = gs2.best_estimator_
    evaluation(X_train, X_test, t_train, t_test, dt, 'DT')

In [63]:
run_models(data, 'doc2vec')

/var/folders/q7/lxfgdwq942qdsd3_mlwlm7p00000gn/T/ipykernel_62893/3287934322.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


Fitting 5 folds for each of 10 candidates, totalling 50 fits
===================MNB Performance=====================
95% CI = [ 0.7184251008082508 0.7675626726795227 ]
Train:                precision    recall  f1-score   support

           0       0.74      1.00      0.85       827
           1       0.00      0.00      0.00       286

    accuracy                           0.74      1113
   macro avg       0.37      0.50      0.43      1113
weighted avg       0.55      0.74      0.63      1113

Test:                precision    recall  f1-score   support

           0       0.72      1.00      0.84       201
           1       0.00      0.00      0.00        78

    accuracy                           0.72       279
   macro avg       0.36      0.50      0.42       279
weighted avg       0.52      0.72      0.60       279

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/opt/anaconda3/envs/PR_Project/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/PR_Project/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/PR_Project/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

===================DT Performance=====================
95% CI = [ 0.6804533049591012 0.7625003629945669 ]
Train:                precision    recall  f1-score   support

           0       1.00      1.00      1.00       827
           1       1.00      1.00      1.00       286

    accuracy                           1.00      1113
   macro avg       1.00      1.00      1.00      1113
weighted avg       1.00      1.00      1.00      1113

Test:                precision    recall  f1-score   support

           0       0.81      0.85      0.83       201
           1       0.57      0.50      0.53        78

    accuracy                           0.75       279
   macro avg       0.69      0.68      0.68       279
weighted avg       0.74      0.75      0.75       279



In [64]:
run_models(data, 'tfidf')

/var/folders/q7/lxfgdwq942qdsd3_mlwlm7p00000gn/T/ipykernel_62893/3287934322.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


Fitting 5 folds for each of 10 candidates, totalling 50 fits
===================MNB Performance=====================
95% CI = [ 0.7228832636808271 0.7738027080051444 ]
Train:                precision    recall  f1-score   support

           0       0.76      0.98      0.86       827
           1       0.65      0.12      0.20       286

    accuracy                           0.76      1113
   macro avg       0.71      0.55      0.53      1113
weighted avg       0.73      0.76      0.69      1113

Test:                precision    recall  f1-score   support

           0       0.73      0.98      0.84       201
           1       0.56      0.06      0.11        78

    accuracy                           0.72       279
   macro avg       0.64      0.52      0.48       279
weighted avg       0.68      0.72      0.63       279

Fitting 5 folds for each of 36 candidates, totalling 180 fits
===================DT Performance=====================
95% CI = [ 0.7196855791542295 0.76441995495130

In [65]:
run_models(equal, 'doc2vec')

/var/folders/q7/lxfgdwq942qdsd3_mlwlm7p00000gn/T/ipykernel_62893/3287934322.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


Fitting 5 folds for each of 10 candidates, totalling 50 fits
===================MNB Performance=====================
95% CI = [ 0.6635121697952623 0.7527378302047375 ]
Train:                precision    recall  f1-score   support

           0       0.82      0.67      0.74      1593
           1       0.72      0.85      0.78      1607

    accuracy                           0.76      3200
   macro avg       0.77      0.76      0.76      3200
weighted avg       0.77      0.76      0.76      3200

Test:                precision    recall  f1-score   support

           0       0.80      0.68      0.74       407
           1       0.71      0.82      0.77       393

    accuracy                           0.75       800
   macro avg       0.76      0.75      0.75       800
weighted avg       0.76      0.75      0.75       800

Fitting 5 folds for each of 36 candidates, totalling 180 fits
===================DT Performance=====================
95% CI = [ 0.7227608177074024 0.76286418229259

In [66]:
run_models(equal, 'tfidf')

/var/folders/q7/lxfgdwq942qdsd3_mlwlm7p00000gn/T/ipykernel_62893/3287934322.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


Fitting 5 folds for each of 10 candidates, totalling 50 fits
===================MNB Performance=====================
95% CI = [ 0.7239073216747745 0.7460926783252257 ]
Train:                precision    recall  f1-score   support

           0       0.78      0.68      0.73      1593
           1       0.72      0.81      0.76      1607

    accuracy                           0.74      3200
   macro avg       0.75      0.74      0.74      3200
weighted avg       0.75      0.74      0.74      3200

Test:                precision    recall  f1-score   support

           0       0.76      0.69      0.72       407
           1       0.71      0.78      0.74       393

    accuracy                           0.73       800
   macro avg       0.73      0.73      0.73       800
weighted avg       0.74      0.73      0.73       800

Fitting 5 folds for each of 36 candidates, totalling 180 fits
===================DT Performance=====================
95% CI = [ 0.6663834146682985 0.71611658533170